In [2]:
#importing library
import numpy as np
import pandas as pd
import plotly.express as px
import re
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
#loading the dataset
df=pd.read_csv('/kaggle/input/google-play-store-apps/googleplaystore.csv')
#showing the datsset
df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10838,Parkinson Exercices FR,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,Medical,"January 20, 2017",1.0,2.2 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [5]:
#getting information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [6]:
#preprocssing the dataset
unneeded_columns=['App','Genres','Current Ver','Android Ver']
df=df.drop(unneeded_columns,axis=1)

In [7]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,"January 7, 2018"
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,"January 15, 2018"
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,"August 1, 2018"
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,"June 8, 2018"
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,"June 20, 2018"
...,...,...,...,...,...,...,...,...,...
10836,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,"July 25, 2017"
10837,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,"July 6, 2018"
10838,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,"January 20, 2017"
10839,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,"January 19, 2015"


In [8]:
#checking for null values in the columns
df.isna().sum()

Category             0
Rating            1474
Reviews              0
Size                 0
Installs             0
Type                 1
Price                0
Content Rating       1
Last Updated         0
dtype: int64

In [13]:
# creating two function one is getting unique and other get_categorical_columns
def get_uniques(df,columns):
    return {column:list(df[column].unique()) for column in columns}
def get_categorical_columns(df):
    return [column for column in df.columns if df.dtypes[column]=='object']

In [14]:
get_uniques(df,get_categorical_columns(df))

{'Category': ['ART_AND_DESIGN',
  'AUTO_AND_VEHICLES',
  'BEAUTY',
  'BOOKS_AND_REFERENCE',
  'BUSINESS',
  'COMICS',
  'COMMUNICATION',
  'DATING',
  'EDUCATION',
  'ENTERTAINMENT',
  'EVENTS',
  'FINANCE',
  'FOOD_AND_DRINK',
  'HEALTH_AND_FITNESS',
  'HOUSE_AND_HOME',
  'LIBRARIES_AND_DEMO',
  'LIFESTYLE',
  'GAME',
  'FAMILY',
  'MEDICAL',
  'SOCIAL',
  'SHOPPING',
  'PHOTOGRAPHY',
  'SPORTS',
  'TRAVEL_AND_LOCAL',
  'TOOLS',
  'PERSONALIZATION',
  'PRODUCTIVITY',
  'PARENTING',
  'WEATHER',
  'VIDEO_PLAYERS',
  'NEWS_AND_MAGAZINES',
  'MAPS_AND_NAVIGATION',
  '1.9'],
 'Reviews': ['159',
  '967',
  '87510',
  '215644',
  '167',
  '178',
  '36815',
  '13791',
  '121',
  '13880',
  '8788',
  '44829',
  '4326',
  '1518',
  '55',
  '3632',
  '27',
  '194216',
  '224399',
  '450',
  '654',
  '7699',
  '61',
  '118',
  '192',
  '20260',
  '203',
  '136',
  '223',
  '1120',
  '227',
  '5035',
  '1015',
  '353',
  '564',
  '8145',
  '36639',
  '158',
  '591',
  '117',
  '176',
  '295221',


In [15]:
df=df.drop(10472,axis=0).reset_index(drop=True)

In [16]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated
0,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,"January 7, 2018"
1,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,"January 15, 2018"
2,ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,"August 1, 2018"
3,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,"June 8, 2018"
4,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,"June 20, 2018"
...,...,...,...,...,...,...,...,...,...
10835,FAMILY,4.5,38,53M,"5,000+",Free,0,Everyone,"July 25, 2017"
10836,FAMILY,5.0,4,3.6M,100+,Free,0,Everyone,"July 6, 2018"
10837,MEDICAL,NaN,3,9.5M,"1,000+",Free,0,Everyone,"January 20, 2017"
10838,BOOKS_AND_REFERENCE,4.5,114,Varies with device,"1,000+",Free,0,Mature 17+,"January 19, 2015"


In [17]:
#changing the datatype
df['Reviews']=df['Reviews'].astype(np.float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [47]:
df['Reviews']

0           159.0
1           967.0
2         87510.0
3        215644.0
4           967.0
           ...   
10835        38.0
10836         4.0
10837         3.0
10838       114.0
10839    398307.0
Name: Reviews, Length: 10840, dtype: float64

In [20]:
(df['Size']=='Varies with device').sum()

1695

In [29]:
df['Size']=df['Size'].apply(lambda x:np.NaN if x=='Varies with device' else x)

In [35]:
df['Size']=df['Size'].apply(lambda x: np.float(x.replace('M',''))*1e6 if type(x)!=float and "M" in x else x)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [36]:
df['Size']=df['Size'].apply(lambda x: np.float(x.replace('k',''))*1e6 if type(x)!=float and "k" in x else x)

In [38]:
#converting data to np.float

df['Size'].astype(np.float)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


0        19000000.0
1        14000000.0
2         8700000.0
3        25000000.0
4         2800000.0
            ...    
10835    53000000.0
10836     3600000.0
10837     9500000.0
10838           NaN
10839    19000000.0
Name: Size, Length: 10840, dtype: float64

In [40]:
#checking is data
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated
0,ART_AND_DESIGN,4.1,159.0,19000000.0,"10,000+",Free,0,Everyone,"January 7, 2018"
1,ART_AND_DESIGN,3.9,967.0,14000000.0,"500,000+",Free,0,Everyone,"January 15, 2018"
2,ART_AND_DESIGN,4.7,87510.0,8700000.0,"5,000,000+",Free,0,Everyone,"August 1, 2018"
3,ART_AND_DESIGN,4.5,215644.0,25000000.0,"50,000,000+",Free,0,Teen,"June 8, 2018"
4,ART_AND_DESIGN,4.3,967.0,2800000.0,"100,000+",Free,0,Everyone,"June 20, 2018"
...,...,...,...,...,...,...,...,...,...
10835,FAMILY,4.5,38.0,53000000.0,"5,000+",Free,0,Everyone,"July 25, 2017"
10836,FAMILY,5.0,4.0,3600000.0,100+,Free,0,Everyone,"July 6, 2018"
10837,MEDICAL,NaN,3.0,9500000.0,"1,000+",Free,0,Everyone,"January 20, 2017"
10838,BOOKS_AND_REFERENCE,4.5,114.0,NaN,"1,000+",Free,0,Mature 17+,"January 19, 2015"


In [46]:
#replacing the , and + with empty string
df['Installs'].apply(lambda x:np.float(x.replace(',','').replace('+','')))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


0           10000.0
1          500000.0
2         5000000.0
3        50000000.0
4          100000.0
            ...    
10835        5000.0
10836         100.0
10837        1000.0
10838        1000.0
10839    10000000.0
Name: Installs, Length: 10840, dtype: float64

In [50]:
df['Price']=df['Price'].apply(lambda x:np.float(x.replace('$','')))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.


In [51]:
df['Price']

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10835    0.0
10836    0.0
10837    0.0
10838    0.0
10839    0.0
Name: Price, Length: 10840, dtype: float64

In [53]:
get_uniques(df,get_categorical_columns(df))

{'Category': ['ART_AND_DESIGN',
  'AUTO_AND_VEHICLES',
  'BEAUTY',
  'BOOKS_AND_REFERENCE',
  'BUSINESS',
  'COMICS',
  'COMMUNICATION',
  'DATING',
  'EDUCATION',
  'ENTERTAINMENT',
  'EVENTS',
  'FINANCE',
  'FOOD_AND_DRINK',
  'HEALTH_AND_FITNESS',
  'HOUSE_AND_HOME',
  'LIBRARIES_AND_DEMO',
  'LIFESTYLE',
  'GAME',
  'FAMILY',
  'MEDICAL',
  'SOCIAL',
  'SHOPPING',
  'PHOTOGRAPHY',
  'SPORTS',
  'TRAVEL_AND_LOCAL',
  'TOOLS',
  'PERSONALIZATION',
  'PRODUCTIVITY',
  'PARENTING',
  'WEATHER',
  'VIDEO_PLAYERS',
  'NEWS_AND_MAGAZINES',
  'MAPS_AND_NAVIGATION'],
 'Installs': ['10,000+',
  '500,000+',
  '5,000,000+',
  '50,000,000+',
  '100,000+',
  '50,000+',
  '1,000,000+',
  '10,000,000+',
  '5,000+',
  '100,000,000+',
  '1,000,000,000+',
  '1,000+',
  '500,000,000+',
  '50+',
  '100+',
  '500+',
  '10+',
  '1+',
  '5+',
  '0+',
  '0'],
 'Type': ['Free', 'Paid', nan],
 'Content Rating': ['Everyone',
  'Teen',
  'Everyone 10+',
  'Mature 17+',
  'Adults only 18+',
  'Unrated'],
 'Las

In [55]:
df[df['Type'].isna()]

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated
9148,FAMILY,NaN,0.0,NaN,0,NaN,0.0,Everyone 10+,"June 28, 2018"


In [57]:
#dropping the 9148
df=df.drop(9148,axis=0).reset_index(drop=True)

In [58]:
#checking for missing values
df.isna().sum()

Category             0
Rating            1473
Reviews              0
Size              1694
Installs             0
Type                 0
Price                0
Content Rating       0
Last Updated         0
dtype: int64

In [60]:
#replacing with mean of the column
df['Rating']=df['Rating'].fillna(df['Rating'].mean())

In [61]:
#replacing with mean of missing value in size column
df['Size']=df['Size'].fillna(df['Size'].mean())

In [62]:
df.isna().sum()

Category          0
Rating            0
Reviews           0
Size              0
Installs          0
Type              0
Price             0
Content Rating    0
Last Updated      0
dtype: int64

In [63]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated
0,ART_AND_DESIGN,4.100000,159.0,1.900000e+07,"10,000+",Free,0.0,Everyone,"January 7, 2018"
1,ART_AND_DESIGN,3.900000,967.0,1.400000e+07,"500,000+",Free,0.0,Everyone,"January 15, 2018"
2,ART_AND_DESIGN,4.700000,87510.0,8.700000e+06,"5,000,000+",Free,0.0,Everyone,"August 1, 2018"
3,ART_AND_DESIGN,4.500000,215644.0,2.500000e+07,"50,000,000+",Free,0.0,Teen,"June 8, 2018"
4,ART_AND_DESIGN,4.300000,967.0,2.800000e+06,"100,000+",Free,0.0,Everyone,"June 20, 2018"
...,...,...,...,...,...,...,...,...,...
10834,FAMILY,4.500000,38.0,5.300000e+07,"5,000+",Free,0.0,Everyone,"July 25, 2017"
10835,FAMILY,5.000000,4.0,3.600000e+06,100+,Free,0.0,Everyone,"July 6, 2018"
10836,MEDICAL,4.191757,3.0,9.500000e+06,"1,000+",Free,0.0,Everyone,"January 20, 2017"
10837,BOOKS_AND_REFERENCE,4.500000,114.0,3.703544e+07,"1,000+",Free,0.0,Mature 17+,"January 19, 2015"


In [76]:
df['Month']=df['Last Updated'].apply(lambda x:re.search('^[^\s]+',x).group(0))

In [77]:
df['Year']=df['Last Updated'].apply(lambda x:re.search('[^\s]+$',x).group(0))

In [80]:
df=df.drop('Last Updated',axis=1)

In [81]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,ART_AND_DESIGN,4.100000,159.0,1.900000e+07,"10,000+",Free,0.0,Everyone,2018,January
1,ART_AND_DESIGN,3.900000,967.0,1.400000e+07,"500,000+",Free,0.0,Everyone,2018,January
2,ART_AND_DESIGN,4.700000,87510.0,8.700000e+06,"5,000,000+",Free,0.0,Everyone,2018,August
3,ART_AND_DESIGN,4.500000,215644.0,2.500000e+07,"50,000,000+",Free,0.0,Teen,2018,June
4,ART_AND_DESIGN,4.300000,967.0,2.800000e+06,"100,000+",Free,0.0,Everyone,2018,June
...,...,...,...,...,...,...,...,...,...,...
10834,FAMILY,4.500000,38.0,5.300000e+07,"5,000+",Free,0.0,Everyone,2017,July
10835,FAMILY,5.000000,4.0,3.600000e+06,100+,Free,0.0,Everyone,2018,July
10836,MEDICAL,4.191757,3.0,9.500000e+06,"1,000+",Free,0.0,Everyone,2017,January
10837,BOOKS_AND_REFERENCE,4.500000,114.0,3.703544e+07,"1,000+",Free,0.0,Mature 17+,2015,January


In [82]:
label_encoder=LabelEncoder()
df['Category']=label_encoder.fit_transform(df['Category'])
category_mappings={index:label for index,label in enumerate(label_encoder.classes_)}

In [83]:
category_mappings

{0: 'ART_AND_DESIGN',
 1: 'AUTO_AND_VEHICLES',
 2: 'BEAUTY',
 3: 'BOOKS_AND_REFERENCE',
 4: 'BUSINESS',
 5: 'COMICS',
 6: 'COMMUNICATION',
 7: 'DATING',
 8: 'EDUCATION',
 9: 'ENTERTAINMENT',
 10: 'EVENTS',
 11: 'FAMILY',
 12: 'FINANCE',
 13: 'FOOD_AND_DRINK',
 14: 'GAME',
 15: 'HEALTH_AND_FITNESS',
 16: 'HOUSE_AND_HOME',
 17: 'LIBRARIES_AND_DEMO',
 18: 'LIFESTYLE',
 19: 'MAPS_AND_NAVIGATION',
 20: 'MEDICAL',
 21: 'NEWS_AND_MAGAZINES',
 22: 'PARENTING',
 23: 'PERSONALIZATION',
 24: 'PHOTOGRAPHY',
 25: 'PRODUCTIVITY',
 26: 'SHOPPING',
 27: 'SOCIAL',
 28: 'SPORTS',
 29: 'TOOLS',
 30: 'TRAVEL_AND_LOCAL',
 31: 'VIDEO_PLAYERS',
 32: 'WEATHER'}

In [84]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,0,4.100000,159.0,1.900000e+07,"10,000+",Free,0.0,Everyone,2018,January
1,0,3.900000,967.0,1.400000e+07,"500,000+",Free,0.0,Everyone,2018,January
2,0,4.700000,87510.0,8.700000e+06,"5,000,000+",Free,0.0,Everyone,2018,August
3,0,4.500000,215644.0,2.500000e+07,"50,000,000+",Free,0.0,Teen,2018,June
4,0,4.300000,967.0,2.800000e+06,"100,000+",Free,0.0,Everyone,2018,June
...,...,...,...,...,...,...,...,...,...,...
10834,11,4.500000,38.0,5.300000e+07,"5,000+",Free,0.0,Everyone,2017,July
10835,11,5.000000,4.0,3.600000e+06,100+,Free,0.0,Everyone,2018,July
10836,20,4.191757,3.0,9.500000e+06,"1,000+",Free,0.0,Everyone,2017,January
10837,3,4.500000,114.0,3.703544e+07,"1,000+",Free,0.0,Mature 17+,2015,January


In [85]:
df['Installs'].unique()

array(['10,000+', '500,000+', '5,000,000+', '50,000,000+', '100,000+',
       '50,000+', '1,000,000+', '10,000,000+', '5,000+', '100,000,000+',
       '1,000,000,000+', '1,000+', '500,000,000+', '50+', '100+', '500+',
       '10+', '1+', '5+', '0+'], dtype=object)

In [88]:
df['Content Rating'].unique()

array(['Everyone', 'Teen', 'Everyone 10+', 'Mature 17+',
       'Adults only 18+', 'Unrated'], dtype=object)

In [91]:
df['Month'].unique()

array(['January', 'August', 'June', 'March', 'April', 'September', 'July',
       'October', 'November', 'May', 'December', 'February'], dtype=object)

In [93]:
install_orderings=['0+','1+','5+','10+','50+','100+','500+','1,000+','5,000+','10,000+','50,000+','100,000+', '500,000+','1,000,000+','5,000,000+','10,000,000+','50,000,000+','100,000,000+','1,000,000,000+','500,000,000+']

In [89]:
rating_order=['Everyone','Everyone 10+','Teen','Mature 17+','Adults only 18+','Unrated']

In [92]:
month_orderings=['January','February','March','April','May','June','July','August','September','October','November','December']

In [107]:
#creating function to binary and ordinal encoding

def binary_encode(df,column,positive_value):
    df=df.copy()
    df[column]=df[column].apply(lambda x:1 if x==positive_value else 0)
    return df
def ordinal_encode(df,column,orderings):
    df=df.copy()
    df[column]=df[column].apply(lambda x:orderings.index(x))
    return df

In [100]:
df=binary_encode(df,'Type','Paid')

In [101]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,0,4.100000,159.0,1.900000e+07,"10,000+",0,0.0,Everyone,2018,January
1,0,3.900000,967.0,1.400000e+07,"500,000+",0,0.0,Everyone,2018,January
2,0,4.700000,87510.0,8.700000e+06,"5,000,000+",0,0.0,Everyone,2018,August
3,0,4.500000,215644.0,2.500000e+07,"50,000,000+",0,0.0,Teen,2018,June
4,0,4.300000,967.0,2.800000e+06,"100,000+",0,0.0,Everyone,2018,June
...,...,...,...,...,...,...,...,...,...,...
10834,11,4.500000,38.0,5.300000e+07,"5,000+",0,0.0,Everyone,2017,July
10835,11,5.000000,4.0,3.600000e+06,100+,0,0.0,Everyone,2018,July
10836,20,4.191757,3.0,9.500000e+06,"1,000+",0,0.0,Everyone,2017,January
10837,3,4.500000,114.0,3.703544e+07,"1,000+",0,0.0,Mature 17+,2015,January


In [111]:
df=ordinal_encode(df,'Content Rating',rating_order)

In [113]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,0,4.100000,159.0,1.900000e+07,9,0,0.0,0,2018,0
1,0,3.900000,967.0,1.400000e+07,12,0,0.0,0,2018,0
2,0,4.700000,87510.0,8.700000e+06,14,0,0.0,0,2018,7
3,0,4.500000,215644.0,2.500000e+07,16,0,0.0,2,2018,5
4,0,4.300000,967.0,2.800000e+06,11,0,0.0,0,2018,5
...,...,...,...,...,...,...,...,...,...,...
10834,11,4.500000,38.0,5.300000e+07,8,0,0.0,0,2017,6
10835,11,5.000000,4.0,3.600000e+06,5,0,0.0,0,2018,6
10836,20,4.191757,3.0,9.500000e+06,7,0,0.0,0,2017,0
10837,3,4.500000,114.0,3.703544e+07,7,0,0.0,3,2015,0


In [114]:
df

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,0,4.100000,159.0,1.900000e+07,9,0,0.0,0,2018,0
1,0,3.900000,967.0,1.400000e+07,12,0,0.0,0,2018,0
2,0,4.700000,87510.0,8.700000e+06,14,0,0.0,0,2018,7
3,0,4.500000,215644.0,2.500000e+07,16,0,0.0,2,2018,5
4,0,4.300000,967.0,2.800000e+06,11,0,0.0,0,2018,5
...,...,...,...,...,...,...,...,...,...,...
10834,11,4.500000,38.0,5.300000e+07,8,0,0.0,0,2017,6
10835,11,5.000000,4.0,3.600000e+06,5,0,0.0,0,2018,6
10836,20,4.191757,3.0,9.500000e+06,7,0,0.0,0,2017,0
10837,3,4.500000,114.0,3.703544e+07,7,0,0.0,3,2015,0


In [116]:
#Spliting and Scaling the Data
y=df['Category']
y

0         0
1         0
2         0
3         0
4         0
         ..
10834    11
10835    11
10836    20
10837     3
10838    18
Name: Category, Length: 10839, dtype: int64

In [118]:
x=df.drop('Category',axis=1)
x

,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,4.100000,159.0,1.900000e+07,9,0,0.0,0,2018,0
1,3.900000,967.0,1.400000e+07,12,0,0.0,0,2018,0
2,4.700000,87510.0,8.700000e+06,14,0,0.0,0,2018,7
3,4.500000,215644.0,2.500000e+07,16,0,0.0,2,2018,5
4,4.300000,967.0,2.800000e+06,11,0,0.0,0,2018,5
...,...,...,...,...,...,...,...,...,...
10834,4.500000,38.0,5.300000e+07,8,0,0.0,0,2017,6
10835,5.000000,4.0,3.600000e+06,5,0,0.0,0,2018,6
10836,4.191757,3.0,9.500000e+06,7,0,0.0,0,2017,0
10837,4.500000,114.0,3.703544e+07,7,0,0.0,3,2015,0


In [120]:
#scaling the data
scaler=StandardScaler()
x=pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

In [121]:
x

,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Year,Month
0,-0.191598,-0.151664,-0.198597,-0.416350,-0.282293,-0.064419,-0.464329,0.545311,-2.103008
1,-0.609215,-0.151388,-0.253654,0.343171,-0.282293,-0.064419,-0.464329,0.545311,-2.103008
2,1.061256,-0.121828,-0.312015,0.849518,-0.282293,-0.064419,-0.464329,0.545311,0.611870
3,0.643638,-0.078063,-0.132528,1.355865,-0.282293,-0.064419,1.850693,0.545311,-0.163809
4,0.226020,-0.151388,-0.376983,0.089997,-0.282293,-0.064419,-0.464329,0.545311,-0.163809
...,...,...,...,...,...,...,...,...,...
10834,0.643638,-0.151705,0.175793,-0.669524,-0.282293,-0.064419,-0.464329,-0.363038,0.224030
10835,1.687682,-0.151717,-0.368173,-1.429045,-0.282293,-0.064419,-0.464329,0.545311,0.224030
10836,0.000000,-0.151717,-0.303206,-0.922698,-0.282293,-0.064419,-0.464329,-0.363038,-2.103008
10837,0.643638,-0.151679,0.000000,-0.922698,-0.282293,-0.064419,3.008204,-2.179735,-2.103008


In [123]:
#spliting between train and test
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)

In [124]:
print(x_train,x_test,y_train,y_test)

         Rating   Reviews      Size  Installs      Type     Price  \
8139   0.000000 -0.151718 -0.305408 -1.935392  3.542421  0.311137   
8809  -0.191598 -0.148072 -0.275677  0.596344 -0.282293 -0.064419   
8939   0.000000 -0.151712  0.076690 -0.922698 -0.282293 -0.064419   
9654   0.000000 -0.151718  0.000000 -1.429045 -0.282293 -0.064419   
8239   0.643638 -0.151707  0.076690 -0.922698 -0.282293 -0.064419   
...         ...       ...       ...       ...       ...       ...   
10538 -0.400407 -0.151695 -0.381387 -0.416350  3.542421  0.123045   
4831  -2.906113 -0.151580 -0.132528 -0.163177 -0.282293 -0.064419   
4856   0.852447 -0.065265  0.682321  1.102692 -0.282293 -0.064419   
72     0.017211 -0.058842 -0.231631  1.102692 -0.282293 -0.064419   
10210 -0.191598 -0.128484 -0.364870  1.102692 -0.282293 -0.064419   

       Content Rating      Year     Month  
8139        -0.464329 -2.179735  0.999710  
8809        -0.464329  0.545311 -0.551649  
8939        -0.464329 -0.363038 -2.1030

In [125]:
x.shape

(10839, 9)

In [126]:
inputs=tf.keras.Input(shape=(9,))
x=tf.keras.layers.Dense(64,activation='relu')(inputs)
x=tf.keras.layers.Dense(64,activation='relu')(x)
outputs=tf.keras.layers.Dense(33,activation='softmax')(x)


2022-12-17 14:02:08.967884: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [129]:
model=tf.keras.Model(inputs=inputs,outputs=outputs)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
batch_size=64
epochs=100
history=model.fit(x_train,y_train,validation_split=0.2,batch_size=batch_size,
                 epochs=epochs,callbacks=[tf.keras.callbacks.ReduceLROnPlateau()],
                 verbose=0)

2022-12-17 14:06:06.342487: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [131]:
#Results
fig=px.line(history.history,
           y=['loss','val_loss'],
           labels={'index':'Epoch','value':'Loss'},
            title='Training and Validation Loss')
fig.show()

In [132]:
model.evaluate(x_test,y_test)

102/102 [==============================] - 0s 2ms/step - loss: 2.6576 - accuracy: 0.2663


[2.657623291015625, 0.26629766821861267]

In [133]:
1/33

0.030303030303030304